# Consolide data with the feature extraction data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Build the train dataset 

In [2]:
df_train_dementia = pd.read_excel(
    '../hokuto_profile.xlsx', sheet_name='dementia')
df_train_dementia['label'] = 'dementia'

df_train_mci = pd.read_excel('../hokuto_profile.xlsx', sheet_name='mci')
df_train_mci['label'] = 'mci'

df_train_control = pd.read_excel(
    '../hokuto_profile.xlsx', sheet_name='control')
df_train_control['label'] = 'control'

# union of the three dataframes
df_train = pd.concat([df_train_dementia, df_train_mci, df_train_control])


df_train['label_id'] = df_train['label'].map(
    {'dementia': 2, 'mci': 1, 'control': 0})
df_train['site_id'] = df_train['Site'].map(
    {'A': 0, 'B': 1})


df_train.head()

,ID,Age,Gender,MMSE,Site,label,label_id,site_id
0,hokuto_dementia1,60,M,13,A,dementia,2,0
1,hokuto_dementia2,64,M,17,B,dementia,2,1
2,hokuto_dementia3,69,F,9,A,dementia,2,0
3,hokuto_dementia4,70,M,22,B,dementia,2,1
4,hokuto_dementia5,73,M,18,A,dementia,2,0


In [3]:
df_features_test = pd.read_csv('extracted_features.csv')
# remove file extension pour filename
df_features_test['FileName'] = df_features_test['FileName'].str.replace(
    '.mat', '')
df_features_test.head()

,FileName,Delta-1:4,Theta-4:8,Alpha-8:13,Beta-13:30,Gamma-30:40
0,hokuto_dementia1,1527.7380,1400.7176,1223.7844,3281.0699,1531.1306
1,hokuto_dementia10,4465.7692,3885.3643,3085.4350,5508.2384,1505.5075
2,hokuto_dementia11,0.0000,11281.5938,0.0000,20694.7154,17422.9477
3,hokuto_dementia12,9232.7663,8138.7951,6629.8026,13220.6634,3823.2126
4,hokuto_dementia13,6805.6846,5951.8738,4787.9760,9408.0489,2752.0021


In [4]:
# merge the two dataframes with FileName and ID
df_train = pd.merge(df_train, df_features_test, left_on='ID',
                    right_on='FileName').drop('FileName', axis=1)
df_train['gender_id'] = df_train['Gender'].map({'M': 1, 'F': 0})
# save csv
df_train.to_csv('df_train.csv', index=False)
df_train.head()

,ID,Age,Gender,MMSE,Site,label,label_id,site_id,Delta-1:4,Theta-4:8,Alpha-8:13,Beta-13:30,Gamma-30:40,gender_id
0,hokuto_dementia1,60,M,13,A,dementia,2,0,1527.7380,1.400718e+03,1223.7844,3.281070e+03,1.531131e+03,1
1,hokuto_dementia2,64,M,17,B,dementia,2,1,0.0000,1.470078e+06,0.0000,2.326050e+06,1.262857e+06,1
2,hokuto_dementia3,69,F,9,A,dementia,2,0,1278.3960,1.214818e+03,1133.0750,3.814643e+03,1.928315e+03,0
3,hokuto_dementia4,70,M,22,B,dementia,2,1,0.0000,3.975737e+05,0.0000,6.295413e+05,3.441654e+05,1
4,hokuto_dementia5,73,M,18,A,dementia,2,0,9715.4041,8.743269e+03,7358.8373,1.555337e+04,3.254478e+03,1


# Build the test dataset

In [5]:
df_test = pd.read_csv('test_extracted_features.csv')
df_test['FileName'] = df_test['FileName'].str.replace('.mat', '')
df_test.head()

,FileName,Delta-1:4,Theta-4:8,Alpha-8:13,Beta-13:30,Gamma-30:40
0,hokuto_test1,2.564830e+07,2.522521e+07,2.451852e+07,8.550627e+07,4.428985e+07
1,hokuto_test10,3.879899e+03,3.625605e+03,3.244233e+03,8.351222e+03,2.515782e+03
2,hokuto_test11,0.000000e+00,4.652632e+05,0.000000e+00,7.361336e+05,4.009662e+05
3,hokuto_test12,1.761727e+03,1.575399e+03,1.328131e+03,3.414823e+03,1.726068e+03
4,hokuto_test13,2.806736e+03,2.639766e+03,2.407759e+03,7.234731e+03,3.125113e+03


In [6]:

df_profile = pd.read_excel('../hokuto_profile.xlsx', sheet_name='test_data')
df_profile['gender_id'] = df_profile['Gender'].map({'M': 1, 'F': 0})
df_profile['site_id'] = df_train['Site'].map(
    {'A': 0, 'B': 1})
df_profile['label'] = ''
df_profile['label_id'] = ''
df_profile.head()

,ID,Age,Gender,Site,gender_id,site_id,label,label_id
0,hokuto_test1,64,M,A,1,0,,
1,hokuto_test2,70,F,A,0,1,,
2,hokuto_test3,61,F,B,0,0,,
3,hokuto_test4,83,F,B,0,1,,
4,hokuto_test5,71,M,A,1,0,,


In [7]:
# merge the two dataframes with FileName and ID
df_test = pd.merge(df_profile, df_test, left_on='ID',
                      right_on='FileName').drop('FileName', axis=1)
df_test.to_csv('df_test.csv', index=False)

df_test.head()

,ID,Age,Gender,Site,gender_id,site_id,label,label_id,Delta-1:4,Theta-4:8,Alpha-8:13,Beta-13:30,Gamma-30:40
0,hokuto_test1,64,M,A,1,0,,,2.564830e+07,2.522521e+07,2.451852e+07,8.550627e+07,4.428985e+07
1,hokuto_test2,70,F,A,0,1,,,2.838066e+03,2.804487e+03,2.718521e+03,8.439152e+03,2.834072e+03
2,hokuto_test3,61,F,B,0,0,,,0.000000e+00,1.525385e+04,0.000000e+00,2.620135e+04,1.867067e+04
3,hokuto_test4,83,F,B,0,1,,,0.000000e+00,2.093090e+04,0.000000e+00,3.685960e+04,2.663569e+04
4,hokuto_test5,71,M,A,1,0,,,3.368586e+02,3.468851e+02,3.629394e+02,1.620425e+03,1.151548e+03
